In [1]:
%pylab notebook
from __future__ import print_function
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio
from charistools.modelEnv import ModelEnv
from netCDF4 import Dataset
import xarray as xr

Populating the interactive namespace from numpy and matplotlib


In [9]:
#%cd /projects/brodzik/derived_hypsometries/MODSCAG_GF_v09_fromFile
#%ls

/projects/brodzik/derived_hypsometries/MODSCAG_GF_v09_fromFile
AM_OBJECTID100/             BR_OBJECTID33/              IN_v01_OBJECTID20/
AM_OBJECTID62/              BR_OBJECTID34/              IN_v01_OBJECTID21/
AM_OBJECTID64/              BR_OBJECTID35/              IN_v01_OBJECTID23/
AM_OBJECTID65/              BR_OBJECTID36/              IN_v01_OBJECTID24/
AM_OBJECTID66/              BR_OBJECTID37/              IN_v01_OBJECTID25/
AM_OBJECTID68/              BR_OBJECTID38/              IN_v01_OBJECTID26/
AM_OBJECTID69/              BR_OBJECTID39/              IN_v01_OBJECTID27/
AM_OBJECTID72/              BR_OBJECTID4/               IN_v01_OBJECTID29/
AM_OBJECTID77/              BR_OBJECTID40/              IN_v01_OBJECTID30/
AM_OBJECTID78/              BR_OBJECTID41/              IN_v01_OBJECTID31/
AM_OBJECTID79/              BR_OBJECTID42/              IN_v01_OBJECTID32/
AM_OBJECTID80/              BR_OBJECTID43/              IN_v01_OBJECTID33/
AM_OBJECTID81/              BR_OBJECT

In [31]:
configFile = "/projects/brodzik/charis_ti_melt/modelEnv_config.ini"
myEnv = ModelEnv(tileConfigFile=configFile, topDir='/projects/brodzik')  

In [32]:
def get_snow_area(drainageID, year, verbose=False):
    ablation_method='grsize_scag'
    threshold='fromFile'
    modice_nstrikes=3
    inputs = myEnv.model_inputs(
        drainageID, 
        year, 
        ablation_method, 
        threshold, 
        modice_nstrikes=modice_nstrikes, 
        verbose=verbose)
    
    all = pd.DataFrame()
    all['SOL_km2'] = inputs["snow_on_land_by_elevation_hyps"].data_by_doy()
    all['EGI_km2'] = inputs["exposed_glacier_ice_by_elevation_hyps"].data_by_doy()
    all['SOI_km2'] = inputs["snow_on_ice_by_elevation_hyps"].data_by_doy()
    
    if verbose:
        print("Read drainageID=%s, year=%d" % (drainageID, year))
    
    return all

In [33]:
def get_snow_for_drainage(drainageID, start_year=2001, stop_year=2014, verbose=False):
    all = pd.DataFrame()
    for year in np.arange(start_year, stop_year+1):
        next = get_snow_area(drainageID, year, verbose=verbose)
        all = pd.concat([all, next])
        
    return all        

In [40]:
# Generate a list of OBJECTID basins by major drainage,
# and aggregate the SOI/SOL/EGI snow cover 
%cd /projects/brodzik/derived_hypsometries/MODSCAG_GF_v09_fromFile/
majorBasinIDs = ['AM', 'BR','GA_v01', 'IN_v01', 'SY_v01']
start_year = 2001
stop_year = 2014
for id in majorBasinIDs:
    ids = glob.glob("%s_OBJECTID*" % id)
    ids.sort()
    print("There are %d sub-basins for the %s major basin" % (len(ids), id))
    majorBasin_sca = pd.DataFrame()
    for drainageID in ids:
        drainage_sca = get_snow_for_drainage(drainageID, start_year=start_year,
                                             stop_year=stop_year)
        drainage_sca.fillna(value=0.0, inplace=True)
        print("drainage = %s" % drainageID)
        print(drainage_sca.sum())
        if majorBasin_sca.empty:
            print("Starting new sca sum for major basin = %s..." % id)
            majorBasin_sca = drainage_sca.copy()
        else:
            print("Adding drainage = %s sca..." % drainageID)
            majorBasin_sca = majorBasin_sca + drainage_sca
    outfile = "%s.SCA.%d-%d.nc" % (id, start_year, stop_year)
    
    print("Accumulated sca:")
    print(majorBasin_sca.sum())
    sca_xr = majorBasin_sca.to_xarray()
    sca_xr.attrs['basins'] = ", ".join(ids)
    sca_xr.to_netcdf(outfile, mode='w', format='NETCDF4')
    print("Saved accumulated SCA to: %s" % outfile)


/projects/brodzik/derived_hypsometries/MODSCAG_GF_v09_fromFile
There are 27 sub-basins for the AM major basin
drainage = AM_OBJECTID100
SOL_km2    30857436.74
EGI_km2      250520.90
SOI_km2     9016865.29
dtype: float64
Starting new sca sum for major basin = AM...
drainage = AM_OBJECTID62
SOL_km2    0.0
EGI_km2    0.0
SOI_km2    0.0
dtype: float64
Adding drainage = AM_OBJECTID62 sca...
drainage = AM_OBJECTID64
SOL_km2    0.0
EGI_km2    0.0
SOI_km2    0.0
dtype: float64
Adding drainage = AM_OBJECTID64 sca...
drainage = AM_OBJECTID65
SOL_km2    42135.08
EGI_km2        0.00
SOI_km2        0.00
dtype: float64
Adding drainage = AM_OBJECTID65 sca...
drainage = AM_OBJECTID66
SOL_km2    18321097.41
EGI_km2      141720.96
SOI_km2     3163072.05
dtype: float64
Adding drainage = AM_OBJECTID66 sca...
drainage = AM_OBJECTID68
SOL_km2    3082.05
EGI_km2       0.00
SOI_km2       0.00
dtype: float64
Adding drainage = AM_OBJECTID68 sca...
drainage = AM_OBJECTID69
SOL_km2    183558.57
EGI_km2         0.

In [13]:
test = saved.to_xarray()
test.attrs['label'] = "this is a label"
test.to_netcdf('AM.SCA.2001-2014.nc')

In [35]:
fromdisk = xr.open_dataset('AM.SCA.2001-2014.nc')
fromdisk

<xarray.Dataset>
Dimensions:  (Date: 5113)
Coordinates:
  * Date     (Date) datetime64[ns] 2001-01-01 2001-01-02 2001-01-03 ...
Data variables:
    SOL_km2  (Date) float64 1.05e+04 1.033e+04 1.022e+04 1.019e+04 1.016e+04 ...
    EGI_km2  (Date) float64 52.16 37.39 32.08 29.21 25.23 27.8 25.69 26.82 ...
    SOI_km2  (Date) float64 1.817e+03 1.812e+03 1.813e+03 1.829e+03 1.85e+03 ...
Attributes:
    basins:   AM_OBJECTID100AM_OBJECTID62AM_OBJECTID64AM_OBJECTID65AM_OBJECTI...

In [37]:
fromdisk.close()

In [36]:
ds = fromdisk.to_dataframe()
ds.sum()

EGI_km2      250520.90
SOI_km2     9016865.29
SOL_km2    30857436.74
dtype: float64

In [ ]:
print(hunza.ix['2014-12-23'], obj0.ix['2014-12-23'], )

In [ ]:
all = hunza + obj0
all.sum()

In [ ]:
all

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(all['SOL'], label='SOL')
ax.plot(all['SOL'] + all['EGI'], label='EGI')
ax.plot(all['SOL'] + all['EGI'] + all['SOI'], label='SOI')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])


In [ ]:
new = pd.DataFrame()
if new.empty:
    print("nothing there")
    

In [ ]:
all['SOL'] = sol
all['EGI'] = egi
all['SOI'] = soi
all

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.plot(all0['SOL'], label='SOL OBJ0')
ax.plot(all0['SOL'] + all0['EGI'], label='EGI')
ax.plot(all0['SOL'] + all0['EGI'] + all0['SOI'], label='SOI')

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])


In [ ]:
mod10 = mod10_cube.read(1)
print(mod10.shape)
np.amin(mod10), np.amax(mod10)

In [ ]:
f = Dataset(mod10a1_filename, 'r', 'HDF5')
d = f.groups['500m'].variables['fsca']

In [ ]:
d

In [ ]:
forig = Dataset(modscag_filename, 'r', 'HDF5')
dorig = forig.groups['500m'].variables['fsca']
dorig